In [16]:
import pandas as pd  
import os

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Importing and cleaning files before mapping

In [5]:
folder_path = '/Users/brandonrhee/Desktop/zipcode_analysis/Data'

In [6]:
# four different data sets used to consolidate into one data set mapping zip codes with another
file_path1 = os.path.join(folder_path, 'household_size.csv')
file_path3 = os.path.join(folder_path, 'zip_county.xlsx')
file_path4 = os.path.join(folder_path, 'median_income.csv')
file_path5 = os.path.join(folder_path, 'income_limits.xlsx')

In [7]:
#imported zip_county and utilizing only zip and county columns
zip_county = pd.read_excel(file_path3, dtype ={'zip':'string', 'county':'string'}, usecols = ['zip', 'county'])

In [8]:
#split ZCTA from zip code 
household_size = pd.read_csv(file_path1, usecols = ['NAME', 'S1101_C01_002E'], converters={'NAME': lambda s: s.split(' ')[1]})
#drop first row
household_size.drop(0, axis = 0, inplace=True)
#rename cols for continuity 
household_size.rename(columns = {'NAME':'zip','S1101_C01_002E':'avg_size'}, inplace=True)
#removed rows where there was null median income 
household_size = household_size[household_size['avg_size'] != '-']
# converted avg size to float 
household_size = household_size.astype({'avg_size': 'float'})
#rounded avg_size
household_size['avg_size'] = household_size['avg_size'].apply(lambda n: round(n))

In [9]:
#split ZCTA from zip code 
median_income = pd.read_csv(file_path4, usecols=['NAME', 'S1901_C01_012E'], dtype = {'S1901_C01_012E': 'object'}, converters={'NAME': lambda s: s.split(' ')[1]})
#dropped first row 
median_income.drop(0, axis = 0, inplace=True)
#renamed columns for continuity 
median_income.rename(columns = {'NAME':'zip','S1901_C01_012E':'median_income'}, inplace=True)

In [10]:
#used columns that have 'lim80' or columns that have 'fips2010'. 
#Original dataset has income limits for very and extremely low income
income_limit = pd.read_excel(file_path5, usecols=lambda col: 'Lim80' in col or col == 'fips2010', converters={'fips2010': lambda s: s[:5]})
#removed all columns that have 'lim80_22'
income_limit.rename(columns=lambda col: col.replace('Lim80_22', '') if col != 'fips2010' else 'county', inplace=True)
#group by counties and averge. This is to account for regions that have multiple zip codes and income limits but the same county 
income_limit = income_limit.groupby('county').mean()

In [11]:
#merge median_income to zip_county by using inner join on zip column
merged_data = pd.merge(zip_county, median_income, how='inner', on=['zip'])
#merged household_size to merged data dataframe by using inner join on zip column 
merged_data = pd.merge(merged_data, household_size, how='inner', on=['zip'])
#merged income limit to merged_data by inner join on county zip column 
merged_data = pd.merge(merged_data, income_limit, how='inner', on=['county'])

In [20]:
#copying merged_w_limit to tmp df
#Using deep = True, modifications to the data or indices of the copy will not be reflected in the original object
tmp = merged_data.copy(deep=True)

#function to retrieve income limit by household size. 
def get_income_limit_for_household(row):
    try:
        col_name = f"p{row.avg_size}"
        return row[col_name]
    except KeyError:
        return '-'
#applying function to temporary table to on 'low_income_limit' col
tmp['low_income_limit'] = tmp.apply(get_income_limit_for_household, axis=1)

#removing rows where 'median_income' & 'low_income_limit' does not have a value
tmp = tmp[tmp['low_income_limit'] != '-']
tmp = tmp[tmp['median_income'] != '-']

merged_w_limit = tmp

In [22]:
# making a temporary copy of merged_w_limit to retrieve records with greater than 1 zipcode per income limit 
# this is to capture zipcodes with duplicates but different income limits.  
tmp = merged_w_limit.copy(deep=True)

#group by zipcode and averaged low_income_limit
avg_income_data = tmp.groupby('zip', as_index=False)['low_income_limit'].mean()

#merge avg_income_data on tmp dataframe by inner join on zip. 
x = pd.merge(tmp, avg_income_data, how='inner', on=['zip'])

#selecting zip, median_income, low_income,limit_y from df x 
x = x[['zip', 'median_income', 'low_income_limit_y']]
x = x.drop_duplicates()

#cleaning median_income column removing commas, '+', '-'
#function to define income status based on median income > income limit 
def get_income_status_from_row(row):
    median_income = row.median_income.replace('+', '').replace(',','').replace('-', '')
    adjusted_income = int(median_income) * .8
    return 'LOW' if row.low_income_limit_y > adjusted_income else 'NORMAL'

x['income_status'] = x.apply(get_income_status_from_row, axis=1)

final_data = x[['zip', 'income_status']]
len(final_data)
#final_data.head()

30620

In [23]:
final_data.head()

,zip,income_status
0,00683,LOW
5,00667,LOW
7,00622,LOW
8,00623,LOW
10,00680,LOW
